# Strong Scaling, static droplet (Xdg, Poisson, steady)

We investigate: How much does the algorithm profit from more rescources for a fixed problem size? In theory the sequential part of the algorithm is the asymtotical limit, which is described by Ahmdal's law.

In [ ]:
//#r "./../../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSE_Solver.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Application.XdgPoisson3;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Parameter sweeps
If you feel the urge to change something stick to this section ...

In [ ]:
int[] core_sweep = {4,8,16,32,64,128,256};
int[] p_sweep    = {2,3,4};
int Problemsize   = 2560000;
int MemoryPerCore = 2500;

## Init Database, Client and Workflowmanager
Set names of database and tables to be written out. 
Names are generated of environment variables (build information of jenkins).
There are defaults though, you see no need change anything.

In [ ]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
string buildNr = System.Environment.GetEnvironmentVariable("BUILD_NUMBER");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "run" : buildname;
string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
buildNr = String.IsNullOrEmpty(buildNr)? thedate : buildNr;

string database_name = String.IsNullOrEmpty(dbname)? "DB_Benchmarks" : dbname;
string table_name = String.Concat(buildname, "_", buildNr);

Client setup and <code>\#SBATCH</code> configuration:
- <code>-N</code> (nodes),
- <code>-C</code> (Processor architecture),
- <code>--mem-per-cpu</code> (allocated memory per core).
<br> Note: <code>--mem-per-cpu</code> must be set that the job is accepted by Lichtenberg scheduler.

In [ ]:
var myBatch = GetDefaultQueue();
if(myBatch is SlurmClient){
    (myBatch as SlurmClient).AdditionalBatchCommands = new string[]{"#SBATCH -C avx512", "#SBATCH --mem-per-cpu=2000"};
} GetDefaultQueue()

In [ ]:
string WFlowName = table_name;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

Project name is set to 'run_10-1-2022'.
Opening existing database 'W:\work\scratch\jw52xeqa\run_10-1-2022'.


In [ ]:
var pair = myBatch.AllowedDatabasesPaths.Pick(0);
string DBName = @"\"+database_name;
string localpath=pair.LocalMountPath+DBName;
string remotepath=pair.PathAtRemote+DBName;
var myDB = OpenOrCreateDatabase(localpath);

Opening existing database 'W:\work\scratch\jw52xeqa\DB_XdgPoisson'.


## Generate Grid
- Domain (-1,1)x(-1,1)x(-1,1)
- equidistant cells, resolution is chosen according to Res

In [ ]:
// 3D domain boundaries
double xMax = 1.0, yMax = 1.0, zMax = 1.0;
double xMin = -1.0, yMin = -1.0, zMin = -1.0;

Some auxiliary methods: 
- calculate N_p (DOF of a scalar variable in 3D),
- predefined partitioning

In [ ]:
public Func<double[],int> GetPartFunc(int cores){
    Func<double[], int> MakeMyPartioning = delegate (double[] X) {
    double x  = X[0];
    double y  = X[1];
    double z  = X[2];

    int sx = 1;
    int sy = 1;
    int sz = 1;
    for (int i = 0; i < Math.Log(cores, 2); i++) {
        if (i % 3 == 0)
            sx*= 2;
        else if(i % 3 == 1)
            sy*=2;
        else
            sz*=2;
    }


    double xspan = (xMax - xMin) / sx;
    double yspan = (yMax - yMin) / sy;
    double zspan = (zMax - zMin) / sz;
    int rank     = int.MaxValue;
    int icore    = 0;
    for (int i = 0; i < sx; i++) {
        for (int j = 0; j < sy; j++) {
            for(int k=0;k<sz;k++){
                bool xtrue = x <= xspan * (i + 1) + xMin;
                bool ytrue = y <= yspan * (j + 1) + yMin;
                bool ztrue = z <= zspan * (k + 1) + zMin;
                if (xtrue && ytrue && ztrue) {
                    rank = icore;
                    return rank;
                }
                icore++;
            }
        }
    }

    return rank;
    };
return MakeMyPartioning;
}

public static int Np(int p) {
    return (p*p*p + 6*p*p + 11*p + 6)/6;
}

In [ ]:
// calculate grid size
Func<int,int,int> ResCoreMapping = delegate (int cores, int p){
    double Res    = Math.Pow(Problemsize/Np(p),0.333333333);
    int ResRndup  = (int)Math.Ceiling(Res);
    return ResRndup;
};

In [ ]:
Func<int,int, GridCommons> DefineGrid = delegate (int cores, int Res){
    double[] xNodes = GenericBlas.Linspace(xMin, xMax, Res + 1);
    double[] yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);
    double[] zNodes = GenericBlas.Linspace(zMin, zMax, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string gname = "XdgPoisson-weakscaling";
    
    Console.WriteLine("Creating grid with " + J + " cells.");
    GridCommons g;
    g = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
    g.AddPredefinedPartitioning("predef"+cores, GetPartFunc(cores));
    g.Name = gname;
    g.EdgeTagNames.Add(1, "Dirichlet");   
    g.DefineEdgeTags(delegate (double[] X) {
        byte ret;
        ret = 1;
        return ret;
    });
    return g;
};

Action<GridCommons> SaveGrid = delegate (GridCommons g){
    // there is no way to check for equal predefined partitioning ...
    /*
    var tmp = new List<IGridInfo>();
    foreach(var grid in myDB.Grids){
        try{
            bool IsMatch = grid.Name.Equals(g.Name)
            && grid.NumberOfCells==g.NumberOfCells;
            if(IsMatch) tmp.Add(grid);
        }
        catch(Exception ex) {
            Console.WriteLine(ex.Message);
        }
    }
    if(tmp.Count()>=1){
        Console.WriteLine("Grid found: "+tmp.Pick(0).Name);
        return;
    }
    */
    myDB.SaveGrid(ref g,true);  
};

Auxiliary datatype to map gridID onto controlobjects and control objects onto job settings.

In [ ]:
struct Parameterz{
    public Parameterz(int _Cores, int _Poly, int _Res){
        Cores = _Cores;
        Poly = _Poly;
        Res = _Res;
    }
    public int Cores;
    public int Poly;
    public int Res;
}

In [ ]:
var Grids = new Dictionary<Parameterz,GridCommons>();
foreach(int p in p_sweep){
foreach(int cores in core_sweep){
    int res = ResCoreMapping(cores,p);
    var grid = DefineGrid(cores,res);
    SaveGrid(grid);
    Grids.Add(new Parameterz(cores,p,res),grid);
}
}

Creating grid with 262144 cells.
Grid Edge Tags changed.
Creating grid with 262144 cells.
Grid Edge Tags changed.
Creating grid with 74088 cells.
Grid Edge Tags changed.
Creating grid with 74088 cells.
Grid Edge Tags changed.


## Generate Control object

In [ ]:
int MaxDof = int.MaxValue;
LinearSolverCode[] solver_sweep = new LinearSolverCode[] {
    //LinearSolverCode.classic_pardiso, 
    //LinearSolverCode.classic_mumps, 
    LinearSolverCode.exp_Kcycle_schwarz, 
    //LinearSolverCode.exp_gmres_levelpmg, 
};

In [ ]:
// - - - - - - - - - - - - - - - - - - -
// Initial Values & Boundary conditions
// - - - - - - - - - - - - - - - - - - -
static class InitialValues {
    public static double RHS(double[] X) {
        return -Math.Sin(X[0]);
    }
    
    public static double DirichletBC(double[] X) {
        return 0.0;
    }
    
    public static double NeumannBC(double[] X) {
       if(Math.Abs(X[1] - 1.0) < 1.0e-8 || Math.Abs(X[1] + 1.0) < 1.0e-8)
           return 0;
       if(X.Length > 2 && (Math.Abs(X[2] - 1.0) < 1.0e-8 || Math.Abs(X[2] + 1.0) < 1.0e-8))
           return 0;

       return Math.Cos(10.0);
   }
} InitialValues.RHS(new double[]{ 0,0})

-0

## governing equations

The test problem can be considered as stationary 3 dimensional heat equation with source-term and two phases:

$$\begin{aligned}
- \mu \Delta T  &= f                \text{ in } \Omega \setminus \mathfrak{I} \\
[[T]]  &= 0                \text{ on } \mathfrak{I}                 \\ 
[[\mu \nabla T \cdot \vec{n}_{\mathfrak{I}}]] &= 0                \text{ on } \mathfrak{I}  \\              
T  &= g_\text{Diri}    \text{ on } \Gamma_\mathrm{Diri}    \\
\nabla T \cdot \vec{n}_{ \partial \Omega}   &= g_\text{Neu}  \text{ on } \Gamma_\mathrm{Neu} 
\end{aligned}$$

with a constant diffusion coefficient in each subdomain

$\mu (\vec{x}) = \begin{pmatrix}
\mu_\mathfrak{A}  \text{for } \vec{x} \in \mathfrak{A} \\
\mu_\mathfrak{B}  \text{for } \vec{x} \in \mathfrak{B}
\end{pmatrix}$

In [ ]:
public Func<LinearSolverCode, int, IGridInfo, int, XdgPoisson3Control> CtrlGenerator = delegate (LinearSolverCode solver, int k, IGridInfo grd, int cores) {

    int _Np = Np(k);
    int J  = grd.NumberOfCells;
    //int rnd         = new Random().Next();
    //string caseName = string.Format("J{0}_k{1}_{2}_c{3}_rnd{4}", J, k, solver_name,cores,rnd);
    string caseName = string.Format("J{0}_k{1}_{2}_c{3}", J, k, solver,cores);
    
    Console.WriteLine("setting up: " + caseName);
    Console.WriteLine("based on grid: "+grd.Name);

    var ctrl = new XdgPoisson3Control();
    
    
    ctrl.AlternateDbPaths = new[]{
        new ValueTuple<string,string>(remotepath, ""),
        new ValueTuple<string,string>(localpath, "")
    };
    ctrl.savetodb = true;
    ctrl.GridGuid = grd.ID;
    
    ctrl.FieldOptions.Add("Phi", new FieldOpts() {
         Degree = 2,
         SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    ctrl.FieldOptions.Add("u", new FieldOpts() {
         Degree = k,
         SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });

    ctrl.LinearSolver.SolverCode           = solver;
    ctrl.LinearSolver.NoOfMultigridLevels  = 10;
    ctrl.LinearSolver.TargetBlockSize      = Math.Min(J*_Np-1,10000);
    ctrl.LinearSolver.ConvergenceCriterion = 1e-8;
    
    double radius           = 0.71;
    ctrl.ExcactSolSupported = false;
    ctrl.InitialValues.Add("Phi", new Formula("X => X[0].Pow2()+X[1].Pow2()+X[2].Pow2()-"+radius+".Pow2()"));
    ctrl.MU_A = -1;
    ctrl.MU_B = -1000;
    ctrl.InitialValues.Add("rhs#A", new Formula("X => 1"));
    ctrl.InitialValues.Add("rhs#B", new Formula("X => 1"));
    ctrl.InitialValues.Add("u#A", new Formula ("X => 0"));
    ctrl.InitialValues.Add("u#B", new Formula ("X => 0"));
    ctrl.CutCellQuadratureType = XQuadFactoryHelper.MomentFittingVariants.Saye;
    ctrl.SetDefaultDiriBndCnd  = true;
    //ctrl.TracingNamespaces     = "*";
    
    ctrl.ViscosityMode = XLaplace_Interface.Mode.SIP;
    ctrl.SessionName   = caseName;
    ctrl.ProjectName   = WFlowName;
    //ctrl.GridPartType  = BoSSS.Foundation.Grid.GridPartType.METIS;
    ctrl.GridPartType    = BoSSS.Foundation.Grid.GridPartType.Predefined;
    ctrl.GridPartOptions = "predef"+cores;
    
    ctrl.AgglomerationThreshold = 0.1;
    
    return ctrl;
};

In [ ]:
var controls = new Dictionary<Parameterz,XdgPoisson3Control>();
foreach(LinearSolverCode solver in solver_sweep) {
foreach(var grid in Grids){
    int k = grid.Key.Poly;
    int cores = grid.Key.Cores;
    var grd = grid.Value;
    int _Np = Np(k);
    int J  = grd.NumberOfCells;
    if(J*_Np > MaxDof)
        continue;
    var ctrl =  CtrlGenerator(solver,k,grd,cores);
    controls.Add(grid.Key, ctrl);
}}

setting up: J262144_k2_exp_Kcycle_schwarz_c4
based on grid: XdgPoisson-weakscaling
setting up: J262144_k2_exp_Kcycle_schwarz_c256
based on grid: XdgPoisson-weakscaling
setting up: J74088_k4_exp_Kcycle_schwarz_c4
based on grid: XdgPoisson-weakscaling
setting up: J74088_k4_exp_Kcycle_schwarz_c256
based on grid: XdgPoisson-weakscaling


## Generate & submit jobs

In [ ]:
// Calculate number of nodes, probably necessary in case of memory bottleneck
Func<int, int> Nodes4WeakScale = delegate(int NoOfCores){
    int MemPerNode = 384*1024; //64 GB on old nodes
    return (int)((double)MemoryPerCore / (double)MemPerNode * NoOfCores)+1;
};
Func<int, int> Nodes4StrongScale = delegate(int NoOfCores){
    int corespernode = 96; //24 cores on old nodes
    return (int)((double)NoOfCores / (double)corespernode) +1;
};
Action<int,BatchProcessorClient, List<string>> NodeRegression =  delegate (int cores, BatchProcessorClient thisBatch, List<string> Cmdtmp) {
    int NoOfNodes = Nodes4StrongScale(cores);
    var tmp = Cmdtmp.CloneNonshallow().ToList();
    tmp.Add($"#SBATCH -N {NoOfNodes}");
    (thisBatch as SlurmClient).AdditionalBatchCommands = tmp.ToArray();
};

In [ ]:
var Cmdtmp = (myBatch as SlurmClient).AdditionalBatchCommands.ToList();
foreach(var cc in controls){
    try {
        var ctrl = cc.Value;
        int ncores = cc.Key.Cores;

        Console.WriteLine(" Submitting: " + ctrl.SessionName); 
        var aJob = new Job(ctrl.SessionName, typeof(XdgPoisson3Main));
        aJob.SetControlObject(ctrl);
        aJob.NumberOfMPIProcs         = ncores;
        aJob.ExecutionTime            = "3:00:00";
        aJob.UseComputeNodesExclusive = true;
        if(myBatch is SlurmClient) NodeRegression.Invoke(ncores,myBatch,Cmdtmp);
        aJob.Activate(myBatch);
    } catch (Exception ex){
        Console.WriteLine(ex.Message);
    }
}

 Submitting: J262144_k2_exp_Kcycle_schwarz_c4
Deploying job J262144_k2_exp_Kcycle_schwarz_c4 ... 
Deploying executables and additional files ...
Deployment directory: X:\Deployerie\run_10-1-2022-XdgPoisson32022Jan10_193938
copied 47 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.
24845792

 Submitting: J262144_k2_exp_Kcycle_schwarz_c256
Deploying job J262144_k2_exp_Kcycle_schwarz_c256 ... 
Deploying executables and additional files ...
Deployment directory: X:\Deployerie\run_10-1-2022-XdgPoisson32022Jan10_193952
copied 47 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.
24845803

 Submitting: J74088_k4_exp_Kcycle_schwarz_c4
Deploying job J74088_k4_exp_Kcycle_schwarz_c4 ... 
Deploying executables and additional files ...
Deployment directory: X:\Deployerie\run_10-1-2022-XdgPoisson32022Jan10_194004
copied 47 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.
24845812

 Submitting: J740

In [ ]:
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(5*3600,900)